In [9]:
import urllib.request, urllib.error
slothlib_path = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
slothlib_file = urllib.request.urlopen(slothlib_path)
slothlib_stopwords = [line.decode("utf-8").strip() for line in slothlib_file]
slothlib_stopwords = [ss for ss in slothlib_stopwords if not ss==u'']
import warnings
warnings.filterwarnings("ignore")
import spacy
nlp = spacy.load('ja_ginza')
from parse_5w1h import parse_5w1h
#テストファイル読み込み&下処理
path = "/Users/shota/Documents/itac/test_summary1.txt"
with open(path) as f:
    text = f.read()
text = '金曜日の朝、ケンタッキー州の工業団地で巨大な火事が発生し、当局が被害を封じ込めようと働いたため、その地域に濃い煙が噴出した。市の緊急管理機関のマイク・ワイマー氏によると、ルイビルのゼネラル・エレクトリック・アプライアンス・パークで午前7時少し前に火災が始まりました。 彼は、負傷者や閉じ込められた者の報告はないと言った。 ビデオは煙と明るいオレンジ色の炎の両方を示しました。 消防士は影響を受けた建物の周りの位置を取り、周辺から水を噴霧しました。 ワイマーは、当局が火災の原因を知らなかったとCNNに語り、少なくとも4つの警報が鳴った。 GEのWebサイトによると、ルイビルアプライアンスパークの施設は、米国の製造業を活性化しています。 公園は大きく、34のサッカー場が施設内の倉庫の1つに収まるようになっています。'
text = text.replace('\n','')

print(text)
doc = nlp(text)

金曜日の朝、ケンタッキー州の工業団地で巨大な火事が発生し、当局が被害を封じ込めようと働いたため、その地域に濃い煙が噴出した。市の緊急管理機関のマイク・ワイマー氏によると、ルイビルのゼネラル・エレクトリック・アプライアンス・パークで午前7時少し前に火災が始まりました。 彼は、負傷者や閉じ込められた者の報告はないと言った。 ビデオは煙と明るいオレンジ色の炎の両方を示しました。 消防士は影響を受けた建物の周りの位置を取り、周辺から水を噴霧しました。 ワイマーは、当局が火災の原因を知らなかったとCNNに語り、少なくとも4つの警報が鳴った。 GEのWebサイトによると、ルイビルアプライアンスパークの施設は、米国の製造業を活性化しています。 公園は大きく、34のサッカー場が施設内の倉庫の1つに収まるようになっています。


In [10]:
import networkx as nx

def increment_edge (graph, node0, node1):
    print("link {} {}".format(node0, node1))
    
    if graph.has_edge(node0, node1):
        graph[node0][node1]["weight"] += 1.0
    else:
        graph.add_edge(node0, node1, weight=1.0)
POS_KEPT = ["ADJ", "NOUN", "PROPN", "VERB"]

def link_sentence (doc, sent, lemma_graph, seen_lemma):
    visited_tokens = []
    visited_nodes = []

    for i in range(sent.start, sent.end):
        token = doc[i]

        if token.pos_ in POS_KEPT and token.lemma_ not in slothlib_stopwords:
            #token.lemma_は原型. token.pos_は品詞
            key = (token.lemma_, token.pos_)

            if key not in seen_lemma:
                seen_lemma[key] = set([token.i])
            else:
                seen_lemma[key].add(token.i)

            node_id = list(seen_lemma.keys()).index(key)

            if not node_id in lemma_graph:
                lemma_graph.add_node(node_id)

            print("visit {} {}".format(visited_tokens, visited_nodes))
            print("range {}".format(list(range(len(visited_tokens) - 1, -1, -1))))
            
            for prev_token in range(len(visited_tokens) - 1, -1, -1):
                print("prev_tok {} {}".format(prev_token, (token.i - visited_tokens[prev_token])))
                
                if (token.i - visited_tokens[prev_token]) <= 3:
                    increment_edge(lemma_graph, node_id, visited_nodes[prev_token])
                else:
                    break

            print(" -- {} {} {} {} {} {}".format(token.i, token.text, token.lemma_, token.pos_, visited_tokens, visited_nodes))

            visited_tokens.append(token.i)
            visited_nodes.append(node_id)

In [11]:
lemma_graph = nx.Graph()
seen_lemma = {}

for sent in doc.sents:
    link_sentence(doc, sent, lemma_graph, seen_lemma)
    #break # only test one sentence
labels = {}
keys = list(seen_lemma.keys())

for i in range(len(seen_lemma)):
    labels[i] = keys[i][0].lower()
ranks = nx.pagerank(lemma_graph)
imp_list = []
for node_id, rank in sorted(ranks.items(), key=lambda x: x[1], reverse=True):
    imp_list.append(labels[node_id])

visit [] []
range []
 -- 0 金曜日 金曜日 NOUN [] []
visit [0] [0]
range [0]
prev_tok 0 2
link 1 0
 -- 2 朝 朝 NOUN [0] [0]
visit [0, 2] [0, 1]
range [1, 0]
prev_tok 1 2
link 2 1
prev_tok 0 4
 -- 4 ケンタッキー州 ケンタッキー州 PROPN [0, 2] [0, 1]
visit [0, 2, 4] [0, 1, 2]
range [2, 1, 0]
prev_tok 2 2
link 3 2
prev_tok 1 4
 -- 6 工業 工業 NOUN [0, 2, 4] [0, 1, 2]
visit [0, 2, 4, 6] [0, 1, 2, 3]
range [3, 2, 1, 0]
prev_tok 3 1
link 4 3
prev_tok 2 3
link 4 2
prev_tok 1 5
 -- 7 団地 団地 NOUN [0, 2, 4, 6] [0, 1, 2, 3]
visit [0, 2, 4, 6, 7] [0, 1, 2, 3, 4]
range [4, 3, 2, 1, 0]
prev_tok 4 2
link 5 4
prev_tok 3 3
link 5 3
prev_tok 2 5
 -- 9 巨大 巨大 ADJ [0, 2, 4, 6, 7] [0, 1, 2, 3, 4]
visit [0, 2, 4, 6, 7, 9] [0, 1, 2, 3, 4, 5]
range [5, 4, 3, 2, 1, 0]
prev_tok 5 2
link 6 5
prev_tok 4 4
 -- 11 火事 火事 NOUN [0, 2, 4, 6, 7, 9] [0, 1, 2, 3, 4, 5]
visit [0, 2, 4, 6, 7, 9, 11] [0, 1, 2, 3, 4, 5, 6]
range [6, 5, 4, 3, 2, 1, 0]
prev_tok 6 2
link 7 6
prev_tok 5 4
 -- 13 発生 発生 VERB [0, 2, 4, 6, 7, 9, 11] [0, 1, 2, 3, 4, 5, 6]
visit [0

In [12]:
import math
def collect_5w1hphrases (chunk,chunk_start,chunk_end, phrases, counts):
    chunk_len = chunk_end - chunk_start + 1
    sq_sum_rank = 0.0
    non_lemma = 0
    compound_key = set([])

    for i in range(chunk_start, chunk_end):
        token = doc[i]
        key = (token.lemma_, token.pos_)
        
        if key in seen_lemma:
            node_id = list(seen_lemma.keys()).index(key)
            rank = ranks[node_id]
            sq_sum_rank += rank
            compound_key.add(key)
        
            #print(" {} {} {} {}".format(token.lemma_, token.pos_, node_id, rank))
        else:
            non_lemma += 1
    
    # although the noun chunking is greedy, we discount the ranks using a
    # point estimate based on the number of non-lemma tokens within a phrase
    non_lemma_discount = chunk_len / (chunk_len + (2.0 * non_lemma) + 1.0)

    # use root mean square (RMS) to normalize the contributions of all the tokens
    phrase_rank = math.sqrt(sq_sum_rank / (chunk_len + non_lemma))
    phrase_rank *= non_lemma_discount

    # remove spurious punctuation
    phrase = chunk.replace("'", "")

    # create a unique key for the the phrase based on its lemma components
    compound_key = tuple(sorted(list(compound_key)))
    
    if not compound_key in phrases:
        phrases[compound_key] = set([ (phrase, phrase_rank) ])
        counts[compound_key] = 1
    else:
        phrases[compound_key].add( (phrase, phrase_rank) )
        counts[compound_key] += 1

    #print("{} {} {} {} {} {}".format(phrase_rank, chunk, chunk_start, chunk_end, chunk_len, counts[compound_key]))

In [13]:
from parse_5w1h import parse_5w1h
parse = parse_5w1h(0)
parse.extract(text)
_5w1h = parse.display_5w1h()

In [17]:
phrases = {}
counts = {}

for i,chunk in enumerate(_5w1h):

     collect_5w1hphrases(chunk.phrase, chunk.start,chunk.end,phrases, counts)

print(phrases)
import operator

min_ranks = {}

for compound_key, rank_tuples in phrases.items():
    l = list(rank_tuples)
    l.sort(key=operator.itemgetter(1), reverse=True)
    
    phrase, rank = l[0]

    min_ranks[phrase] = (rank)

{(('ケンタッキー州', 'PROPN'), ('団地', 'NOUN'), ('工業', 'NOUN'), ('巨大', 'ADJ'), ('朝', 'NOUN'), ('火事', 'NOUN'), ('金曜日', 'NOUN')): {('金曜日の朝、ケンタッキー州の工業団地で巨大な火事が', 0.036120255718123884)}, (('発生', 'VERB'),): {('発生し、', 0.02011301072490084)}, (('当局', 'NOUN'),): {('当局が', 0.03917599845785792)}, (('被害', 'NOUN'),): {('被害を', 0.028122648026814603)}, (('働く', 'VERB'), ('封じ込める', 'VERB')): {('封じ込めようと働いた', 0.029882712234651644)}, (('為', 'NOUN'),): {('ため、', 0.026837964181059604)}, (('地域', 'NOUN'), ('濃い', 'ADJ')): {('その地域に濃い', 0.03239472659266023)}, (('煙', 'NOUN'),): {('煙が', 0.041459448113587344)}, (('噴出', 'VERB'),): {('噴出した。', 0.01476065053376242)}, (('マイク', 'PROPN'), ('機関', 'NOUN'), ('管理', 'NOUN'), ('緊急', 'NOUN')): {('市の緊急管理機関のマイク・', 0.032496758040122004)}, (('よる', 'VERB'), ('ワイマー', 'PROPN'), ('氏', 'NOUN')): {('ワイマー氏によると、', 0.03438693105396105)}, (('アプライアンス', 'NOUN'), ('ゼネラル・エレクトリック', 'PROPN'), ('ルイビル', 'PROPN')): {('ルイビルのゼネラル・エレクトリック・アプライアンス・', 0.035324612060273354)}, (('パーク', 'NOUN'), ('午前', 'NOUN')): {('パークで午

In [15]:
i = 0 
summary = []
while i < len(_5w1h):
    if _5w1h[i].phrase in min_ranks:
        if min_ranks[_5w1h[i].phrase] > 0.035:
            print(_5w1h[i].phrase)
            print(_5w1h[i]._type)
    i += 1
    

金曜日の朝、ケンタッキー州の工業団地で巨大な火事が
Who
当局が
Who
煙が
Who
ルイビルのゼネラル・エレクトリック・アプライアンス・
None
煙と明るいオレンジ色の炎の両方を
What
当局が
Who
火災の原因を
What
ルイビルアプライアンスパークの施設は、
Who


In [8]:
#####要約最新#####
i = 0 
summary = []
count = []
while i < len(_5w1h):
    if _5w1h[i].phrase in min_ranks:
        if min_ranks[_5w1h[i].phrase] > 0.035:
            if _5w1h[i].phrase not in count:
                summary.append(_5w1h[i].phrase)
                count.append(_5w1h[i].phrase)
                
            if _5w1h[i]._type =="How":
                l = i-1
                while l >= 0:
                    if _5w1h[l].phrase not in count:
                        summary.insert(0,_5w1h[l].phrase)
                        count.append(_5w1h[l].phrase)
                    if _5w1h[l]._type == 'Who'or _5w1h._type[l] == 'What':
                        if summary != []:
                            print(''.join(summary))
                        summary = []
                        break
                    l -= 1
            else:
                i = i+1
                while i < len(_5w1h):
                    if _5w1h[i].phrase not in count:
                        summary.append(_5w1h[i].phrase)
                        count.append(_5w1h[i].phrase)
                    if _5w1h[i]._type == 'How' and min_ranks[_5w1h[i+1].phrase] < 0.035:
                        if summary != []:
                            print(''.join(summary))
                        summary = []
                        break
                    i += 1
                
    i += 1

金曜日の朝、ケンタッキー州の工業団地で巨大な火事が発生し、当局が被害を封じ込めようと働いた
煙が噴出した。
ルイビルのゼネラル・エレクトリック・アプライアンス・パークで午前7時少し前に火災が始まりました。
煙と明るいオレンジ色の炎の両方を示しました。
火災の原因を知らなかったとCNNに語り、少なくとも
ルイビルアプライアンスパークの施設は、米国の製造業を活性化しています。
